### Industrial Production

In [1]:
import sys
sys.path.append('../src')

import uschartbook.config

from uschartbook.config import *
from uschartbook.utils import *

In [2]:
url_base = 'https://www.federalreserve.gov/datadownload/Output.aspx?rel=G17'
dates = 'from=01/01/1988&to=12/31/2021'
series = 'series=f97ad3652d87a6c1720943c31423103a'
settings = 'filetype=csv&label=include&layout=seriescolumn'
url = f'{url_base}&{series}&lastobs=&{dates}&{settings}'

d, data = clean_fed_data(url)

data = data.rename(d, axis=1)

(data.loc['1989':, ['Manufacturing', 'Total index']]
 .to_csv(data_dir / 'indpro.csv', index_label='date', float_format='%g'))

print('Latest: ', dtxt(data.index[-1])['mon1'])

Latest:  October 2021


In [3]:
# Retrieve weights
adj_series_dict = {k[3:-2]: v for k, v in d.items()}
series = adj_series_dict.keys()
url = 'https://www.federalreserve.gov/releases/g17/ipdisk/ipweights_sa.txt'
columns = ['Series', 'Year', 'January', 'February', 'March', 
          'April', 'May', 'June', 'July', 'August', 
          'September', 'October', 'November', 'December']
raw_weights = pd.read_csv(url, sep='\s+', skiprows=1)
raw_weights.columns = columns
weights = (raw_weights[raw_weights['Series'].isin(series)]
           .set_index(['Series', 'Year']).stack().reset_index())
weights['Date'] = (pd.to_datetime(weights['level_2'] + ' 01, ' 
                + weights['Year'].astype('int').astype('str')))
weights = (weights.set_index(['Series', 'Date'])[0]
           .unstack().T.rename(adj_series_dict, axis=1)
           .loc['1988':])

weights['ENS'] = weights['Equipment, total'] + weights['Nonindustrial supplies']

In [4]:
# Apply calculations
growth = data.pct_change(12).dropna() * 100
growth['ENS'] = ((growth['Equipment, total'] * (weights['Equipment, total'] / weights['ENS']))
                 + (growth['Nonindustrial supplies'] * (weights['Nonindustrial supplies'] / weights['ENS'])))
contrib = growth / 100 * weights.iloc[12:]

In [5]:
ltmonth = dtxt(data.index[-1])['mon1']
prmonth = dtxt(data.index[-2])['mon1']
tot = inc_dec_percent(growth['Total index'].iloc[-1])
prtot = inc_dec_percent(growth['Total index'].iloc[-2], 'of')
man = growth['Manufacturing'].iloc[-1]
manu = cont_subt(contrib['Manufacturing'].iloc[-1], digits=1)
mining = cont_subt(contrib['Mining'].iloc[-1], digits=1)
egu = cont_subt(contrib['Electric and gas utilities'].iloc[-1], 'end', 1)

cg = cont_subt(contrib['Consumer goods'].iloc[-1], digits=1)
eq = cont_subt(contrib['Equipment, total'].iloc[-1], 'end', 1)
nes = cont_subt(contrib['Nonindustrial supplies'].iloc[-1], 'end', 1)
mat = cont_subt(contrib['Materials'].iloc[-1], 'end', 1)

In [6]:
text = ('A monthly index produced by the Federal Reserve '+
        '\href{https://www.federalreserve.gov/releases/g17/Current/default.htm}{shows}'+
        f' industrial production {tot} over the year ending {ltmonth}, '+
        f'following {prtot} over the year ending {prmonth}. '+
        f'One-year growth in manufacturing production was {man:.1f} '+
        f'percent in {ltmonth}, and manufacturing {manu} the '+
        'overall change in industrial production. Over the same period, '+
        f'mining {mining} the overall change, and electric and gas '+
        f'utilities {egu}. \n\n'+
        f'By market group, production of consumer goods {cg} '+
        f'one-year industrial production growth in {ltmonth}. Production of '+
        f'business equipment {eq}, production of nonindustrial supplies {nes}, '+
        f'and production of materials {mat}.')

write_txt(text_dir / 'indpro.txt', text)

print(text)

A monthly index produced by the Federal Reserve \href{https://www.federalreserve.gov/releases/g17/Current/default.htm}{shows} industrial production increased by 5.1 percent over the year ending October 2021, following an increase of 4.6 percent over the year ending September 2021. One-year growth in manufacturing production was 4.5 percent in October 2021, and manufacturing contributed 3.3 percentage points to the overall change in industrial production. Over the same period, mining contributed 1.7 percentage points to the overall change, and electric and gas utilities contributed 0.4 percentage point. 

By market group, production of consumer goods contributed 0.7 percentage point to one-year industrial production growth in October 2021. Production of business equipment contributed 1.3 percentage points, production of nonindustrial supplies contributed 0.6 percentage point, and production of materials contributed 2.6 percentage points.


In [7]:
date_latest = dtxt(data.index[-1])['datetime']
month_short = data.index[-1].strftime('%b')

text_full = ('\\noindent \hspace*{-2mm} \\begin{tikzpicture}'+
'\\begin{axis}[\\bbar{y}{0}, \dateaxisticks ytick={60, 80, 100}, '+
'enlarge y limits={0.05}, enlarge x limits={0}, legend cell align={left},'+
'yticklabel style={text width=1.0em}, '+
'xtick={{1989-01-01}, {1995-01-01}, {2000-01-01}, {2005-01-01}, '+
f'{{2010-01-01}}, {{2015-01-01}}, {{2020-01-01}}, {{{date_latest}}}}},'+
f'xticklabels={{`89, `95, `00, `05, `10, `15, ,{month_short}}}, '+
'minor xtick={}, '+
'clip=false, height=5.4cm, width=6.8cm,'+
'legend style={fill=white, legend columns=1, at={(1.02, 0.33)}}]'+
'\\rbars'+
'\\thinline{red}{date}{Manufacturing}{data/indpro.csv}'+
'\\thinline{blue!90!black}{date}{Total index}{data/indpro.csv}'+
'\legend{Manufacturing, Total Index};'+
'\end{axis}'+
'\end{tikzpicture}\\')

write_txt(text_dir / 'ip_main_line.tex', text_full)

In [8]:
srs = ['Consumer goods', 'ENS', 'Materials', 'Durable manufacturing', 
       'Mining', 'Nondurable manufacturing', 'Electric and gas utilities']

(contrib[srs].resample('QS').mean()
     .to_csv(data_dir / 'indprogr.csv', index_label='date', float_format='%g'))

(contrib.loc['2015':, srs].to_csv(data_dir / 'indprogr_rec.csv', 
                                       index_label='date', float_format='%g'))

In [9]:
date_latest = data.index[-1].strftime('%Y-%m-%d')
month_short = data.index[-1].strftime('%b')

text = ('yticklabel style={text width=1.5em},'+
        'xtick={{2015-01-01}, {2016-01-01}, {2017-01-01}, '+
        '{2018-01-01}, {2019-01-01}, {2020-01-01}, {2021-01-01}, '+
        f'{{{date_latest}}}}}, '+
        f'xticklabels={{`15, `16, `17, `18, `19, `20, ,{month_short}}}')

text_full = ('\\noindent \hspace*{-2mm} \\begin{tikzpicture}'+
' \\begin{axis}[\\bbar{y}{0}, \dateaxisticks ytick={-15, -10, -5, 0, 5, 10, 15}, '+
' clip=false, width=6.65cm, height=5.5cm, yticklabel style={text width=1.28em}, '+
text + 
', minor xtick={}, enlarge y limits=0.08, '+
' enlarge x limits={0.005}]'+
' \\ctsbar{violet!60!black}{date}{Consumer goods}{data/indprogr_rec.csv}{2.2pt}'+
' \\ctsbar{magenta}{date}{ENS}{data/indprogr_rec.csv}{2.2pt}'+
' \\ctsbar{orange!70!yellow}{date}{Materials}{data/indprogr_rec.csv}{2.2pt}'+
' \end{axis}'+
' \end{tikzpicture}'+
' \hfill'+
' \\begin{tikzpicture}'+
' \\begin{axis}[\\bbar{y}{0}, \dateaxisticks ytick={-15, -10, -5, 0, 5, 10, 15}, '+
' clip=false, width=6.65cm, height=5.5cm, yticklabel style={text width=1.28em}, '+
text + 
', minor xtick={}, enlarge y limits=0.08, '+
' enlarge x limits={0.005}]'+
' \\ctsbar{blue!60!black}{date}{Durable manufacturing}{data/indprogr_rec.csv}{2.2pt}'+
' \\ctsbar{blue!20!cyan!80!white}{date}{Nondurable manufacturing}{data/indprogr_rec.csv}{2.2pt}'+
' \\ctsbar{orange!20!yellow}{date}{Mining}{data/indprogr_rec.csv}{2.2pt}'+
' \\ctsbar{green!80!blue}{date}{Electric and gas utilities}{data/indprogr_rec.csv}{2.2pt}'+
' \end{axis}'+
' \end{tikzpicture}\\')

write_txt(text_dir / 'ip_latest_monthly.tex', text_full)

### IP Table

In [10]:
n = {'Total index': '& \hspace{-1mm}Total index',
     'Manufacturing': '& \hspace{1mm}Manufacturing',
     'Durable manufacturing': '\cbox{blue!60!black} & \hspace{3mm}Durable manufacturing',
     'Motor vehicles and parts': ' & \hspace{5mm}Motor vehicles \& parts',
     'Nondurable manufacturing': '\cbox{blue!20!cyan!80!white} & \hspace{3mm}Nondurable manufacturing',
     'Mining': '\cbox{orange!20!yellow} & \hspace{1mm}Mining',
     'Electric and gas utilities': '\cbox{green!80!blue} & \hspace{1mm}Utilities',
     'Consumer goods': '\cbox{violet!60!black} & \hspace{1mm}Consumer goods',
     'Durable consumer goods': ' & \hspace{3mm}Consumer durables',
     'Automotive products': ' & \hspace{5mm}Automotive products',
     'Nondurable consumer goods': ' & \hspace{3mm}Consumer nondurables',
     'Foods and tobacco': ' & \hspace{5mm}Foods and tobacco',
     'Chemical products': ' & \hspace{5mm}Chemical products',
     'Consumer energy products': ' & \hspace{5mm}Consumer energy products',
     'ENS': '\cbox{magenta} & \hspace{1mm}Equipment \& nonindustrial supplies',
     'Equipment, total': ' & \hspace{3mm}Equipment',
     'Industrial equipment': ' & \hspace{5mm}Industrial equipment',
     'Nonindustrial supplies': ' & \hspace{3mm}Nonindustrial supplies',
     'Construction supplies': ' & \hspace{5mm}Construction supplies',
     'Business supplies': ' & \hspace{5mm}Business supplies',
     'Materials': '\cbox{orange!70!yellow} & \hspace{1mm}Materials',
     'Consumer parts': ' & \hspace{3mm}Consumer parts',
     'Equipment parts': ' & \hspace{3mm}Equipment parts',
     'Chemical materials': ' & \hspace{3mm}Chemical materials',
     'Energy materials': ' & \hspace{3mm}Energy materials'}

table = contrib[n.keys()].iloc[-3:].iloc[::-1].T
table.columns = [dtxt(date)['mon6'] for date in table.columns]
table[dtxt(data.index[-13])['mon6']] = contrib[n.keys()].iloc[-13]
table2 = growth[n.keys()].iloc[-3:].iloc[::-1].T
table2.columns = [' ' + dtxt(date)['mon6'] for date in table2.columns]
table2[' '+dtxt(data.index[-13])['mon6']] = growth[n.keys()].iloc[-13]
table = table.join(table2)

table = table.applymap('{:,.1f}'.format)
table.index = [n[name] for name in table.index]

(table.to_csv(data_dir / 'indpro.tex', sep='&', line_terminator='\\\ ', 
              quotechar=' ', index_label='&'))

### Bar Chart

In [11]:
base = 'https://www.federalreserve.gov/datadownload/Output.aspx?'
srs = 'rel=G17&series=644452cb9b9f8c5a43cd9afb772f1b16&lastobs=50&'
dt = 'from=&to=&'
oth = 'filetype=csv&label=include&layout=seriescolumn'
url = base + srs + dt + oth

d, data = clean_fed_data(url)

ltdate = dtxt(data.index[-1])['mon1']
write_txt(text_dir / 'ip_ind_ldate.txt', ltdate)

df = pd.DataFrame()
for term, name in [('IP', 'IP'), ('CAP.', 'CP')]:
    keys = [key for key in data.keys() if term in key]
    td = (1 - data[keys].iloc[-1] / data[keys].loc['2020-02-01'])
    td.index = td.index.map(d).str.replace('and', '\&')
    df[name] = td * 100
    
final = df.sort_values('IP', ascending=False)

final.round(1).to_csv(data_dir / 'ip_comp.csv', index_label='name', sep=';')
final.index = [i.replace('products', 'product')
                .replace('product', 'products')
                .replace('eq.', 'equipment') for i in final.index]
words = ['none', 'one', 'two', 'three', 'four', 'five', 'six', 'seven', 
         'eight', 'nine', 'ten', '11', 'all 12']
nums = list(range(0, 13))
nums_to_words = dict(zip(nums, words))
verb = {n: 'were' if n!= 1 else 'was' for n in range(0, 13)}

In [12]:
thresh = 0.1
incval = len(final[final['IP'] >= thresh])
decval = len(final[final['IP'] <= -thresh])
unchval = len(final[(final['IP'] < thresh) & (final['IP'] > -thresh)])

text0 = (f'As of {ltdate}, of a subset of 12 industries that contribute the majority of industrial production, '+
        f'{nums_to_words[incval]} increased \\textbf{{production}} since February '+
        f'2020, {nums_to_words[decval]} decreased '+
        f'production, and {nums_to_words[unchval]} {verb[unchval]} unchanged '
        +'(see\cbox{green!60!lime}). ')

largest = final.IP[abs(final.IP).sort_values(ascending=False).iloc[:4].index]
n = {}
i = 0
for name, value in largest.iteritems():
    iname = f'production of {name}' if 'product' in name else f'{name} production'
    inc_dec1 = inc_dec_percent(value)
    inc_dec2 = inc_dec_percent(value, 'of')
    n[i] = (f'{iname} {inc_dec1}'.lower())
    i += 1
text1 = (f'Since February 2020, {n[0]}, {n[1]}, {n[2]}, and {n[3]}. \n\n')

incval = len(final[final['CP'] >= thresh])
decval = len(final[final['CP'] <= -thresh])
unchval = len(final[(final['CP'] < thresh) & (final['CP'] > -thresh)])

text2 = (f'Since February 2020, {nums_to_words[incval]} of the 12 '+
         f'industries increased \\textbf{{capacity}}, {nums_to_words[decval]} '+
         f'decreased capacity, and {nums_to_words[unchval]} {verb[unchval]} unchanged '+
         '(see\cbox{cyan!90!blue}). ')

largest = final.CP[abs(final.CP).sort_values(ascending=False).iloc[:3].index]
n = {}
i = 0
for name, value in largest.iteritems():
    iname = f'production capacity for {name}' if 'product' in name else f'{name} capacity'
    inc_dec1 = inc_dec_percent(value)
    inc_dec2 = inc_dec_percent(value, 'of')
    n[i] = (f'{iname} {inc_dec1}'.lower())
    i += 1
    
text3 = (f'{n[0].capitalize()}, {n[1]}, and {n[2]}.')

end_text = text0 + text1 + text2 + text3

print(end_text)

write_txt(text_dir / 'ip_comp.txt', end_text)

As of October 2021, of a subset of 12 industries that contribute the majority of industrial production, seven increased \textbf{production} since February 2020, five decreased production, and none were unchanged (see\cbox{green!60!lime}). Since February 2020, aerospace \& miscellaneous transportation equipment production decreased by 14.6 percent, production of computer \& electronic products decreased by 10.4 percent, mining production increased by 8.5 percent, and motor vehicles \& parts production increased by 6.5 percent. 

Since February 2020, five of the 12 industries increased \textbf{capacity}, six decreased capacity, and one was unchanged (see\cbox{cyan!90!blue}). Production capacity for computer \& electronic products decreased by 5.3 percent, electric \& gas utilities capacity decreased by 4.6 percent, and mining capacity increased by 1.7 percent.


### Capacity Utilization

In [13]:
base = 'https://www.federalreserve.gov/datadownload/Output.aspx?'
srs = 'rel=G17&series=316680f2d5251c61c995df7ae36b4b07&lastobs=&'
dt = 'from=01/01/1989&to=12/31/2021&'
oth = 'filetype=csv&label=include&layout=seriescolumn'
url = base + srs + dt + oth

d = {'CAPUTL.B00004.S': 'Manufacturing', 'CAPUTL.B50001.S': 'Total index'}

df = pd.read_csv(url, skiprows=5, index_col=0)[d.keys()].rename(d, axis=1)

df.index = pd.to_datetime(df.index)

df.to_csv(data_dir / 'tcu.csv', index_label='date')

node = end_node(df['Total index'], 'blue!80!black')
write_txt(text_dir / 'tcu_tot_node.txt', node)

node = end_node(df['Manufacturing'], 'blue!40!cyan')
write_txt(text_dir / 'tcu_mfg_node.txt', node)

ldate = df.index[-1].strftime("%B %Y")
lval = df['Total index'].iloc[-1]
sval = df.loc['1989-01-01', 'Total index']
tch = sval - lval

text = (f'In {ldate}, the industrial capacity utilization rate was '+
        f'{lval:.1f} percent '+
        '(see {\color{blue!80!black}\\textbf{---}}), '+
        'and the manufacturing capacity utilization rate was '+
        f'{df["Manufacturing"].iloc[-1]:.1f} percent '+
        '(see {\color{blue!40!cyan}\\textbf{---}}). Total capacity '+
        f'utilization has fallen by {tch:.1f} percentage points since '+
        'January 1989.')

write_txt(text_dir / 'tcu.txt', text)
print(text)

In October 2021, the industrial capacity utilization rate was 76.4 percent (see {\color{blue!80!black}\textbf{---}}), and the manufacturing capacity utilization rate was 76.7 percent (see {\color{blue!40!cyan}\textbf{---}}). Total capacity utilization has fallen by 8.9 percentage points since January 1989.
